In [1]:
import pandas as pd
import numpy as np
import re
from string import ascii_letters, digits
import utils.cleaning_dicts
import os

In [2]:
print(utils.cleaning_dicts.clean_team_pff)

{'arz': 'ari', 'blt': 'bal', 'clv': 'cle', 'hst': 'hou', 'la': 'lar', 'sd': 'lac', 'sl': 'lar'}


### Read in and process all pff position datasets

In [39]:
####################################################################################
                ###   Read-in and clean all passing datasets ###
####################################################################################

passing_depth = pd.read_csv('./historic_data/pff_data/passing_depth_hist.csv')
passing_allowed_pressure = pd.read_csv('./historic_data/pff_data/passing_allowed_pressure_hist.csv')
passing_pressure = pd.read_csv('./historic_data/pff_data/passing_pressure_hist.csv')
passing_concept = pd.read_csv('./historic_data/pff_data/passing_concept_hist.csv')
time_in_pocket = pd.read_csv('./historic_data/pff_data/time_in_pocket_hist.csv')
passing_summ_conc = pd.read_csv('./historic_data/pff_data/passing_summ_conc_hist.csv')


def drop_non_qbs(df):
    df=df.rename(columns={"player_id": "numeric_id"})
    df=df[df['position'] == 'QB']
    df['position']=df['position'].astype(str).str.lower()
    df['team_name']=df['team_name'].astype(str).str.lower()       
    df['player']=df['player'].str.replace('[^a-zA-Z0-9]', '').str.lower()
    df['team_name']=df['team_name'].str.lower()
    df['year'] = df['year'].astype(str)
    df['week'] = df['week'].astype(str)
    df.insert(0, "p_id", (df['player']+'_'+df['team_name']+'_'+df['year']+'_'+df['week']))
    df.insert(1, "unique_team_id", (df['team_name']+'_'+df['year']+'_'+df['week']))
    return df
    
passing_depth = drop_non_qbs(passing_depth)
passing_allowed_pressure = drop_non_qbs(passing_allowed_pressure)
passing_pressure = drop_non_qbs(passing_pressure)
passing_concept = drop_non_qbs(passing_concept)
time_in_pocket = drop_non_qbs(time_in_pocket)
passing_summ_conc = drop_non_qbs(passing_summ_conc)


passing_depth = passing_depth[passing_depth.columns.drop(list(passing_depth.filter(regex='left|right|center')))]

####################################################################################
				###   Read-in and clean all receiving datasets ###
####################################################################################

rec_summ_conc = pd.read_csv('./historic_data/pff_data/rec_summ_conc_hist.csv')
receiving_concept = pd.read_csv('./historic_data/pff_data/receiving_concept_hist.csv')
receiving_depth = pd.read_csv('./historic_data/pff_data/receiving_depth_hist.csv')
receiving_scheme = pd.read_csv('./historic_data/pff_data/receiving_scheme_hist.csv')

def drop_non_recs(df):
    df=df.rename(columns={"player_id": "numeric_id"})
    df= df[df.position.str.match('WR|TE|HB|FB')]
    df['position']=df['position'].astype(str).str.lower()
    df['team_name']=df['team_name'].astype(str).str.lower()       
    df['player']=df['player'].str.replace('[^a-zA-Z0-9]', '').str.lower()
    df['team_name']=df['team_name'].str.lower()
    df['year'] = df['year'].astype(str)
    df['week'] = df['week'].astype(str)
    df.insert(0, "p_id", (df['player']+'_'+df['team_name']+'_'+df['year']+'_'+df['week']))
    df.insert(1, "unique_team_id", (df['team_name']+'_'+df['year']+'_'+df['week']))
    return df

rec_summ_conc = drop_non_recs(rec_summ_conc)
receiving_concept = drop_non_recs(receiving_concept)
receiving_depth = drop_non_recs(receiving_depth)
receiving_scheme = drop_non_recs(receiving_scheme)


####################################################################################
				###   Read-in and clean all rushing datasets ###
####################################################################################

rush_summ_conc = pd.read_csv('./historic_data/pff_data/rush_summ_conc_hist.csv')

def drop_non_rbs(df):
    df=df.rename(columns={"player_id": "numeric_id"})
    df= df[df.position.str.match('WR|HB|FB')]
    df['position']=df['position'].astype(str).str.lower()
    df['team_name']=df['team_name'].astype(str).str.lower()       
    df['player']=df['player'].str.replace('[^a-zA-Z0-9]', '').str.lower()
    df['team_name']=df['team_name'].str.lower()
    df['year'] = df['year'].astype(str)
    df['week'] = df['week'].astype(str)
    df.insert(0, "p_id", (df['player']+'_'+df['team_name']+'_'+df['year']+'_'+df['week']))
    df.insert(1, "unique_team_id", (df['team_name']+'_'+df['year']+'_'+df['week']))
    return df

rush_summ_conc = drop_non_rbs(rush_summ_conc)


####################################################################################
				###   Read-in and clean all blocking datasets ###
####################################################################################


block_summ_conc = pd.read_csv('./historic_data/pff_data/block_summ_conc_hist.csv')
offense_pass_blocking = pd.read_csv('./historic_data/pff_data/offense_pass_blocking_hist.csv')
offense_run_blocking = pd.read_csv('./historic_data/pff_data/offense_run_blocking_hist.csv')


def drop_non_ols(df):
    df=df.rename(columns={"player_id": "numeric_id"})
    df = df[df['position'].notna()]
    df= df[df.position.str.match('T|C|G|TE')]
    df['position']=df['position'].astype(str).str.lower()
    df['team_name']=df['team_name'].astype(str).str.lower()       
    df['player']=df['player'].str.replace('[^a-zA-Z0-9]', '').str.lower()
    df['team_name']=df['team_name'].str.lower()
    df['year'] = df['year'].astype(str)
    df['week'] = df['week'].astype(str)
    df.insert(0, "p_id", (df['player']+'_'+df['team_name']+'_'+df['year']+'_'+df['week']))
    df.insert(1, "unique_team_id", (df['team_name']+'_'+df['year']+'_'+df['week']))
    return df


block_summ_conc	= drop_non_ols(block_summ_conc)
offense_pass_blocking = drop_non_ols(offense_pass_blocking)
offense_run_blocking = drop_non_ols(offense_run_blocking)



####################################################################################
				###   Read-in and clean all defensive datasets ###
####################################################################################

def_summ_conc = pd.read_csv('./historic_data/pff_data/def_summ_conc_hist.csv')
pass_rush_summary = pd.read_csv('./historic_data/pff_data/pass_rush_summary_hist.csv')
run_defense_summary = pd.read_csv('./historic_data/pff_data/run_defense_summary_hist.csv')
defense_coverage_scheme = pd.read_csv('./historic_data/pff_data/defense_coverage_scheme_hist.csv')
defense_coverage_summary = pd.read_csv('./historic_data/pff_data/defense_coverage_summary_hist.csv')
slot_coverage = pd.read_csv('./historic_data/pff_data/slot_coverage_hist.csv')


def drop_non_def(df):
    df=df.rename(columns={"player_id": "numeric_id"})
    df['position']=df['position'].astype(str).str.lower()
    df['team_name']=df['team_name'].astype(str).str.lower()       
    df['player']=df['player'].str.replace('[^a-zA-Z0-9]', '').str.lower()
    df['team_name']=df['team_name'].str.lower()
    df['year'] = df['year'].astype(str)
    df['week'] = df['week'].astype(str)
    df.insert(0, "p_id", (df['player']+'_'+df['team_name']+'_'+df['year']+'_'+df['week']))
    df.insert(1, "unique_team_id", (df['team_name']+'_'+df['year']+'_'+df['week']))
    return df

def_summ_conc = drop_non_def(def_summ_conc)
pass_rush_summary = drop_non_def(pass_rush_summary)
run_defense_summary = drop_non_def(run_defense_summary)
defense_coverage_scheme = drop_non_def(defense_coverage_scheme)
defense_coverage_summary = drop_non_def(defense_coverage_summary)
slot_coverage = drop_non_def(slot_coverage)	
	

####################################################################################
				###   Read-in and clean all special teams datasets ###
####################################################################################	

st_kickers = pd.read_csv('./historic_data/pff_data/st_kickers_hist.csv')
st_punters = pd.read_csv('./historic_data/pff_data/st_punters_hist.csv')

def clean_spec(df):
    df=df.rename(columns={"player_id": "numeric_id"})
    df['position']=df['position'].astype(str).str.lower()
    df['team_name']=df['team_name'].astype(str).str.lower()       
    df['player']=df['player'].str.replace('[^a-zA-Z0-9]', '').str.lower()
    df['team_name']=df['team_name'].str.lower()
    df['year'] = df['year'].astype(str)
    df['week'] = df['week'].astype(str)
    df.insert(0, "p_id", (df['player']+'_'+df['team_name']+'_'+df['year']+'_'+df['week']))
    df.insert(1, "unique_team_id", (df['team_name']+'_'+df['year']+'_'+df['week']))
    return df

st_kickers =clean_spec(st_kickers)
st_punters = clean_spec(st_punters)



####################################################################################
####################################################################################
####################################################################################

/home/tomb/anaconda3/envs/cloak/lib/python3.7/site-packages/ipykernel_launcher.py:18: FutureWarning: The default value of regex will change from True to False in a future version.
/home/tomb/anaconda3/envs/cloak/lib/python3.7/site-packages/ipykernel_launcher.py:50: FutureWarning: The default value of regex will change from True to False in a future version.
/home/tomb/anaconda3/envs/cloak/lib/python3.7/site-packages/ipykernel_launcher.py:75: FutureWarning: The default value of regex will change from True to False in a future version.
/home/tomb/anaconda3/envs/cloak/lib/python3.7/site-packages/ipykernel_launcher.py:102: FutureWarning: The default value of regex will change from True to False in a future version.
/home/tomb/anaconda3/envs/cloak/lib/python3.7/site-packages/ipykernel_launcher.py:133: FutureWarning: The default value of regex will change from True to False in a future version.
/home/tomb/anaconda3/envs/cloak/lib/python3.7/site-packages/ipykernel_launcher.py:160: FutureWarni

In [40]:
passing_summ_conc.head(n=5)

,p_id,unique_team_id,player,numeric_id,position,team_name,player_game_count,accuracy_percent,aimed_passes,attempts,...,scrambles,spikes,thrown_aways,touchdowns,turnover_worthy_plays,twp_rate,yards,ypa,week,year
0,ryantannehill_ten_2021_1,ten_2021_1,ryantannehill,7014,qb,ten,1,68.1,47,49,...,1,0,0,1,1,1.6,298,6.1,1,2021
1,jalenhurts_phi_2021_1,phi_2021_1,jalenhurts,40291,qb,phi,1,76.2,42,48,...,4,0,4,2,2,3.3,387,8.1,1,2021
2,tombrady_tb_2021_1,tb_2021_1,tombrady,698,qb,tb,1,67.5,40,43,...,1,0,2,0,0,0.0,269,6.3,1,2021
3,mattryan_atl_2021_1,atl_2021_1,mattryan,4317,qb,atl,1,69.2,39,42,...,1,1,1,4,0,0.0,283,6.7,1,2021
4,matthewstafford_la_2021_1,la_2021_1,matthewstafford,4924,qb,la,1,70.7,41,41,...,2,0,0,2,1,2.1,280,6.8,1,2021


### Add prefixes to all columns.  Creating column names structured as "source-dataset_column-name"

In [13]:
####################################################################################
								###   add prefixes ###
####################################################################################	

def create_prefix(prefix=None, df=None):
    id = df[['p_id']]
    temp = df.drop(['p_id','player','numeric_id','position','team_name'], axis=1)
    temp = temp.add_prefix(prefix)
    id = pd.concat([id, temp], axis=1)
    return id

def id_prefix(prefix=None, df=None):
    id = df[['p_id','player','numeric_id','position','team_name']]
    temp = df.drop(['p_id','player','numeric_id','position','team_name'], axis=1)
    temp = temp.add_prefix(prefix)
    id = pd.concat([id, temp], axis=1)
    return id

passing_summ_conc = id_prefix(prefix="pass_summary_", df=passing_summ_conc)
rush_summ_conc = id_prefix(prefix="rush_summary_", df=rush_summ_conc)
rec_summ_conc = id_prefix(prefix="rec_summary_", df=rec_summ_conc)
block_summ_conc = id_prefix(prefix="block_summary_", df=block_summ_conc)
def_summ_conc = id_prefix(prefix="def_summary_", df=def_summ_conc)
st_kickers = id_prefix(prefix="kicking_", df=st_kickers)
st_punters = id_prefix(prefix="punting_", df=st_punters)


passing_depth = create_prefix(prefix="pass_depth_", df=passing_depth)
passing_allowed_pressure = create_prefix(prefix="pressure_source_", df=passing_allowed_pressure)
passing_pressure = create_prefix(prefix="pass_under_pressure_", df=passing_pressure)
passing_concept = create_prefix(prefix="pass_concept_", df=passing_concept)
time_in_pocket = create_prefix(prefix="pass_time_", df=time_in_pocket)


receiving_concept = create_prefix(prefix="rec_concept_", df=receiving_concept)
receiving_depth = create_prefix(prefix="rec_depth_", df=receiving_depth)
receiving_scheme = create_prefix(prefix="rec_scheme_", df=receiving_scheme)

offense_pass_blocking = create_prefix(prefix="pass_block_", df=offense_pass_blocking)
offense_run_blocking = create_prefix(prefix="run_block_", df=offense_run_blocking)


pass_rush_summary = create_prefix(prefix="pass_rush_", df=pass_rush_summary)
run_defense_summary = create_prefix(prefix="run_defense_", df=run_defense_summary)
defense_coverage_scheme = create_prefix(prefix="def_coverage_scheme_", df=defense_coverage_scheme)
defense_coverage_summary = create_prefix(prefix="def_coverage_summary_", df=defense_coverage_summary)
slot_coverage = create_prefix(prefix="def_slot_coverage_", df=slot_coverage)



### Read in weather data and clean raiders name - merged onto spreads data below ###

In [19]:
### read in weather data###
weather = pd.read_csv('/home/tomb/nfl_models/historic_data/weather_data/weather_hist.csv')

def raiders(df):
    if 'oak' in str(df.away_matchup_id) and '2020' in str(df.away_matchup_id):
        return df.away_matchup_id.replace("oak","lv")
    if 'oak' in str(df.away_matchup_id) and '2021' in str(df.away_matchup_id):
        return df.away_matchup_id.replace("oak","lv")
    if 'oak' in str(df.away_matchup_id) and '2022' in str(df.away_matchup_id):
        return df.away_matchup_id.replace("oak","lv")
    else:
        return df.away_matchup_id
weather['away_matchup_id'] = weather.apply(lambda df: raiders(df), axis=1)

### Create spreads data ###

In [22]:
####################################################################################
				###   spreads data cleaning and engineering ###
####################################################################################

spreads = pd.read_csv('./spreads_data/spreads.csv')

spreads = spreads[spreads['schedule_season']>=2014]
spreads = spreads[['schedule_season','schedule_week','team_home_abb','score_home','score_away','away_team_abb','team_favorite_id','spread_favorite','over_under_line','starting_spread', 'Total Score Open',
       'fav_team_open', 'fav_team_cur', 'remain_fav', 'spread_movement','ou_movement', 'strong_movement', 'fav_team_stronger']]
spreads['team_home_abb'] = spreads['team_home_abb'].astype(str)
spreads['team_favorite_id'] = spreads['team_favorite_id'].astype(str)
spreads['over_under_line'] = spreads['over_under_line'].astype(float)


def fav_spread(nData):
    if nData['team_home_abb'] == nData['team_favorite_id']:
        return nData['spread_favorite']
    elif nData['away_team_abb'] == nData['team_favorite_id']:
        return nData['spread_favorite']
    else:
        pass
spreads['fav_spread'] = spreads.apply(lambda nData: fav_spread(nData), axis=1)

def nonfav_spread(nData):
    if nData['team_home_abb'] != nData['team_favorite_id']:
        return nData['team_home_abb']
    elif nData['away_team_abb'] != nData['team_favorite_id']:
        return nData['away_team_abb']
    else:
        pass
spreads['team_notfav_id'] = spreads.apply(lambda nData: nonfav_spread(nData), axis=1)

def cover_or_not(nData):    
    if nData['team_home_abb'] == nData['team_favorite_id']:
        if ((nData['score_home']-nData['score_away']))+nData['spread_favorite'] > 0:
            return 'Cover'
        elif ((nData['score_home']-nData['score_away']))+nData['spread_favorite'] == 0:            
            return 'Push'       
        else:            
            return 'No Cover'
    elif nData['away_team_abb'] == nData['team_favorite_id']:        
        if ((nData['score_away']-nData['score_home']))+nData['spread_favorite'] > 0:            
            return 'Cover'        
        elif ((nData['score_away']-nData['score_home']))+nData['spread_favorite'] == 0:            
            return 'Push'        
        else:            
            return 'No Cover'
spreads['fav_cover'] = spreads.apply(lambda nData: cover_or_not(nData), axis=1)

def OU_or_not(nData):    
    if (nData['score_home']+nData['score_away']) > nData['over_under_line']:        
        return 'Over'    
    elif (nData['score_home']-nData['score_away']) == nData['over_under_line']:        
        return 'Push'    
    else:        
        return 'Under'
spreads['over_under_result'] = spreads.apply(lambda nData: OU_or_not(nData), axis=1)



spreads['schedule_season'] = spreads['schedule_season'].apply(int)    
spreads['schedule_week'] = spreads['schedule_week'].apply(int)  
data = spreads.sort_values(by=["team_home_abb","schedule_season","schedule_week"], ascending=[True, True, True])

def clean_spreads(df):
    ##  basic scrubbing to clean data ##    
    df['schedule_season'] = df['schedule_season'].apply(str)    
    df['schedule_week'] = df['schedule_week'].apply(str)        
    df=df.apply(lambda x: x.astype(str).str.lower())    
    #df['schedule_week']=df['schedule_week'].astype(str).str[:-2].astype(object)    
    #df['schedule_season'] = df['schedule_season'].astype(str).str[:-2].astype(object)        
    
    ##  create our unique ids  ##
    df.insert(0, "home_matchup_id", (df['team_home_abb']+'vs'+df['away_team_abb']+'_'+df['schedule_season']+'_'+df['schedule_week']))
    df.insert(1, "away_matchup_id", (df['away_team_abb']+'@'+df['team_home_abb']+'_'+df['schedule_season']+'_'+df['schedule_week']))
    df.insert(2, "home_id", (df['team_home_abb']+'_'+df['schedule_season']+'_'+df['schedule_week']))
    df.insert(3, "away_id", (df['away_team_abb']+'_'+df['schedule_season']+'_'+df['schedule_week']))
    return df
    
data = clean_spreads(data)

data = pd.merge(data, weather, on='away_matchup_id', how='left')


sh = data
sa = data

sh = sh.rename(columns={'home_id':'team_id'})
sh.drop('away_id', axis=1, inplace=True)

sa = sa.rename(columns={'away_id':'team_id'})
sa.drop('home_id', axis=1, inplace=True)

spread_comb = pd.concat([sh, sa], axis=0)
spread_comb['team_abb'] = spread_comb['team_id'].astype(str).str[:3]
spread_comb['team_abb'] = spread_comb['team_abb'].str.replace("_","")

def hora1(nData):
    if nData['team_favorite_id'] == nData['team_home_abb']:
        return 1
    elif nData['team_notfav_id'] == nData['team_home_abb']:
        return 1
    else:
        return 0
spread_comb['homeoraway'] = spread_comb.apply(lambda nData: hora1(nData), axis=1)

def hora(nData):
    if nData['team_favorite_id'] == nData['away_team_abb']:
        return 1
    else:
        return 0
spread_comb['fav_homeoraway'] = spread_comb.apply(lambda nData: hora(nData), axis=1)
#sh['fav_homeoraway'] = sh.apply(lambda nData: hora(nData), axis=1)

def ws(nData):
    if (nData['fav_homeoraway'] == 0) & (nData['fav_cover'] == 'cover'):
        return 1
    elif (nData['fav_homeoraway'] == 1) & (nData['fav_cover'] == 'no cover'):
        return 1
    else:
        return 0

def ls(nData):    
    if (nData['fav_homeoraway'] == 0) & (nData['fav_cover'] == 'no cover'):
        return 1
    elif (nData['fav_homeoraway'] == 1) & (nData['fav_cover'] == 'cover'):
        return 1
    else:
        return 0

spread_comb['ats_w'] = spread_comb.apply(lambda nData: ws(nData), axis=1)
spread_comb['ats_l'] = spread_comb.apply(lambda nData: ls(nData), axis=1)
spread_comb.head(n=5).T

,0,1,2,3,4
home_matchup_id,arivslac_2014_1,arivssf_2014_3,arivswas_2014_6,arivsphi_2014_8,arivslar_2014_10
away_matchup_id,lac@ari_2014_1,sf@ari_2014_3,was@ari_2014_6,phi@ari_2014_8,lar@ari_2014_10
team_id,ari_2014_1,ari_2014_3,ari_2014_6,ari_2014_8,ari_2014_10
schedule_season,2014,2014,2014,2014,2014
schedule_week,1,3,6,8,10
team_home_abb,ari,ari,ari,ari,ari
score_home,18,23,30,24,31
score_away,17,14,20,20,14
away_team_abb,lac,sf,was,phi,lar
team_favorite_id,ari,sf,ari,ari,ari


## Read in historic weekly football outsiders data.  Clean raiders name and computer rolling mean###

In [33]:
def raiders_fo(df):
    if 'oak' in str(df.team) and '2020' in str(df.year):
        return df.team_id.str.replace("oak_","lv_")
    elif 'oak' in str(df.team) and '2021' in str(df.year):
        return df.team_id.str.replace("oak_","lv_")
    else:
        return str(df.team_id)

def rolling_fo(data=None, roll_value=None, roll_type=None):
    
    """
        Args:
        data: input pandas dataframe to be rolled
        roll_value: input the number, default is three ## we will need to modify the function if we want more ##
        roll_type: 'mean','std', or 'var' are the only options at the point
        ## assign mean for a given team & year as opposed to the entire dataset
   
    """
    
    data = data.sort_values(by=["team","year","week"], ascending=[True, True, True])
    #data=data.fillna(data.mean())
    num_cols = ['total_dvoa', 'off_dvoa','off_pass_dvoa', 'off_rush_dvoa', 'def_dvoa', 'def_pass_dvoa','def_rush_dvoa', 'special_teams_dvoa']
    ids = data[['team_id', 'year', 'team', 'week', 'opp']].reset_index(drop=True)
   
    if roll_type == 'mean':
        roll3 = data.groupby(['team','year'])[num_cols].apply(lambda x : x.shift().rolling(roll_value).mean())
        roll2 = data.groupby(['team','year'])[num_cols].apply(lambda x : x.shift().rolling(roll_value-1).mean())
        roll1 = data.groupby(['team','year'])[num_cols].apply(lambda x : x.shift().rolling(roll_value-2).mean())
        roll3 = pd.DataFrame(roll3.combine_first(roll2).combine_first(roll1)).reset_index(drop=True)
        df = pd.concat([ids, roll3], axis=1)
    return df

In [38]:
fo = pd.read_csv('./historic_data/fo_data/fo_weekly_hist.csv')

fo['team_id'] = fo.apply(lambda df: raiders_fo(df), axis=1) 
fo_roll = rolling_fo(data=fo, roll_value=3, roll_type='mean')
fo_roll.head().T

,0,1,2,3,4
team_id,ari_2014_1,ari_2014_2,ari_2014_3,ari_2014_5,ari_2014_6
year,2014,2014,2014,2014,2014
team,ari,ari,ari,ari,ari
week,1,2,3,5,6
opp,sd,nyg,sf,den,was
total_dvoa,NaN,0.59,0.625,0.613333,0.56
off_dvoa,NaN,0.53,0.505,0.55,0.533333
off_pass_dvoa,NaN,0.53,0.45,0.516667,0.486667
off_rush_dvoa,NaN,0.51,0.57,0.55,0.56
def_dvoa,NaN,0.61,0.56,0.49,0.406667


### PFF team_game_summaries (tgs) clean and prepare for moving averages (if wanted)

In [132]:
tgs = pd.read_csv('./historic_data/pff_data/team_game_summaries_historic.csv')
tgs = tgs[tgs['year'] >= 2014]

tgs['team_name'] = tgs['team'].map(utils.cleaning_dicts.clean_team_pff_full).fillna(tgs['team'])
tgs['opponent_name'] = tgs['opponent'].map(utils.cleaning_dicts.clean_team_pff_opp).fillna(tgs['opponent'])

tgs['home_or_away']=tgs['home_or_away'].astype(str)

def home_team(nData):
    if str('@') in nData['home_or_away']:
        return nData['opponent_name']
    else:
        return nData['team_name']

tgs['home_team'] = tgs.apply(lambda nData: home_team(nData), axis=1)

def away_team(nData):
    if str('@') in nData['home_or_away']:
        return nData['team_name']
    else:
        return nData['opponent_name']
    
tgs['away_team'] = tgs.apply(lambda nData: away_team(nData), axis=1)

def clean_pff_team_summ(df):
##  basic scrubbing to clean data ##

    df['year'] = df['year'].astype(str)
    df['week'] = df['week'].astype(str)
    df['home_or_away']=np.where(df['home_or_away'] == "@", 1, 0)
    df['wl_int'] = np.where(df['wl'] == "W", 1, 0)
    df=df.replace('-','', regex=True)
    df=df.replace(' ','', regex=True)


    ##  create our unique ids  ##
    df.insert(1, "team_id_impute", (df['team_name']+'_'+df['year']))
    df.insert(2, "opponent_id", (df['opponent_name']+'_'+df['year']+'_'+df['week']))
    df.insert(3, "home_matchup_id", (df['home_team']+'vs'+df['away_team']+'_'+df['year']+'_'+df['week']))

    return df
   
tgs_clean = clean_pff_team_summ(tgs)
tgs_clean.head().T

### Lets create a simple PFF team_game_summary dataset for modeling with averages across all games for a full season

In [150]:
import pandas as pd
import numpy as np

##  Add a column so we can count how many games have been played
tgs_clean['game'] = 1

## subset columns to what we need ##
tgs_sub = tgs_clean[['game','team_name', 'opponent_name', 'home_team', 'away_team','year', 'week', 'wl_int', 'pf', 
               'pa', 'overall_performance']]

## create a point differential variable ##
tgs_sub['point_diff']=tgs_sub.pf-tgs_sub.pa

tgs_sub['year']=tgs_sub['year'].astype(int)
tgs_sub['week']=tgs_sub['week'].astype(int)
## sort columns to prepare for rolling functions ##
tgs_sub = tgs_sub.sort_values(by=["team_name","year","week"], ascending=[True, True, True])


tgs_sub['wl_total'] = tgs_sub.groupby(by=['team_name', 'year'])['wl_int'].transform(lambda x: x.cumsum())
tgs_sub['game_count'] = tgs_sub.groupby(by=['team_name', 'year'])['game'].transform(lambda x: x.cumsum())

def cum_averages(var=None, new_name=None):
    test = tgs_sub.groupby(by=['team_name', 'year'])[var].expanding().mean().reset_index(0)
    test.reset_index(inplace=True)
    test.set_index('level_1', inplace=True)
    test = test[[var]]
    test.columns = [new_name]
    return pd.concat([tgs_sub, test], axis=1)
    
## calculate averages for pf (points_for), pa (points_Against), point_diff, and overall pff performance ###
## we can add all the pff columns from the TGS dataset but this is just an example ##
tgs_sub = cum_averages(var='pf', new_name='avg_pf')
tgs_sub = cum_averages(var='pa', new_name='avg_pa')
tgs_sub = cum_averages(var='point_diff', new_name='avg_point_diff')
tgs_sub = cum_averages(var='overall_performance', new_name='pff_ovr_total')

tgs_sub['wl_total']=tgs_sub['wl_total'].astype(int)
tgs_sub['game_count']=tgs_sub['game_count'].astype(int)
tgs_sub['win_percentage'] = (tgs_sub['wl_total']/tgs_sub['game_count'])

## create some ids so we van find the oppositions stats ##
tgs_sub['year']=tgs_sub['year'].astype(str)
tgs_sub['week']=tgs_sub['week'].astype(str)
tgs_sub['team_id']= tgs_sub.team_name.str.cat(tgs_sub.year, sep="_").str.cat(tgs_sub.week, sep="_")
tgs_sub['matchup_id']= tgs_sub.opponent_name.str.cat(tgs_sub.year, sep="_").str.cat(tgs_sub.week, sep="_")

## create an opponent dataframe where we will merge on their TGS stats ##
tgs_sub['year']=tgs_sub['year'].astype(int)
tgs_sub['week']=tgs_sub['week'].astype(int)




def id_prefix(prefix=None, df=None):
    id = df[['team_id','team_name', 'opponent_name', 'home_team', 'away_team', 'year','week','matchup_id']]
    temp = df.drop(['team_id','team_name', 'opponent_name', 'home_team', 'away_team', 'year','week','matchup_id'], axis=1)
    temp = temp.add_prefix(prefix)
    id = pd.concat([id, temp], axis=1)
    return id
    
opp_df = id_prefix(prefix="opp_", df=tgs_sub)

## subset opponent dataset and merge onto the tgs_Sub ##
opp_df = opp_df[['team_id', 'opp_wl_total', 'opp_game_count', 'opp_avg_pf', 'opp_avg_pa', 'opp_avg_point_diff', 'opp_pff_ovr_total', 'opp_win_percentage']]


tgs_sub.wl_total = tgs_sub.wl_total.shift(1)
tgs_sub.game_count = tgs_sub.game_count.shift(1)
tgs_sub.avg_pf = tgs_sub.avg_pf.shift(1)
tgs_sub.avg_pa = tgs_sub.avg_pa.shift(1)
tgs_sub.avg_point_diff = tgs_sub.avg_point_diff.shift(1)
tgs_sub.pff_ovr_total = tgs_sub.pff_ovr_total.shift(1)
tgs_sub.win_percentage = tgs_sub.win_percentage.shift(1)


opp_df.opp_wl_total = opp_df.opp_wl_total.shift(1)
opp_df.opp_game_count = opp_df.opp_game_count.shift(1)
opp_df.opp_avg_pf = opp_df.opp_avg_pf.shift(1)
opp_df.opp_avg_pa = opp_df.opp_avg_pa.shift(1)
opp_df.opp_avg_point_diff = opp_df.opp_avg_point_diff.shift(1)
opp_df.opp_pff_ovr_total = opp_df.opp_pff_ovr_total.shift(1)
opp_df.opp_win_percentage = opp_df.opp_win_percentage.shift(1)

final = pd.merge(tgs_sub, opp_df, left_on='matchup_id', right_on='team_id', how='left')

final['year']=final['year'].astype(str)
final['week']=final['week'].astype(str)
final['home_matchup_id']= final.home_team.str.cat(final.away_team, sep="vs").str.cat(final.year, sep="_").str.cat(final.week, sep="_")


final.head().T

/home/tomb/anaconda3/envs/cloak/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/home/tomb/anaconda3/envs/cloak/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/tomb/anaconda3/envs/cloak/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value

,0,1,2,3,4
game,1,1,1,1,1
team_name,ari,ari,ari,ari,ari
opponent_name,lac,nyg,sf,den,was
home_team,ari,nyg,ari,den,ari
away_team,lac,ari,sf,ari,was
year,2014,2014,2014,2014,2014
week,1,2,3,5,6
wl_int,1,1,1,0,1
pf,18,25,23,20,30
pa,17,14,14,41,20


### Lets create a simple PFF team_game_summary dataset for modeling with averages across all games for a full season.

### Now lets grab the columns we need from the final TGS dataframe and add the spreads data ###

In [151]:
tgs_model = final[['home_matchup_id','home_team', 'away_team', 'year',
       'week', 'wl_total', 'game_count', 'avg_pf', 'avg_pa', 'avg_point_diff',
       'pff_ovr_total', 'win_percentage','opp_wl_total', 'opp_game_count', 'opp_avg_pf',
       'opp_avg_pa', 'opp_avg_point_diff', 'opp_pff_ovr_total',
       'opp_win_percentage']]

tgs_model['week']=tgs_model['week'].astype(int)
tgs_model= tgs_model[tgs_model['week'] >= 3]

spreads_for_tgs = spread_comb[['home_matchup_id','fav_homeoraway','team_favorite_id','fav_cover','spread_favorite',
       'over_under_line', 'starting_spread', 'Total Score Open', 'spread_movement',
       'ou_movement', 'strong_movement', 'fav_team_stronger', 'fav_spread','precip', 'dome',
       'temperature', 'wind_mph']]

tgs_model = pd.merge(tgs_model, spreads_for_tgs, on='home_matchup_id', how='left')
tgs_model.drop_duplicates(inplace=True)
tgs_model.tail(n=10).T

/home/tomb/anaconda3/envs/cloak/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,7188,7190,7192,7194,7196,7198,7200,7202,7204,7206
home_matchup_id,denvswas_2021_8,wasvstb_2021_10,carvswas_2021_11,wasvssea_2021_12,lvvswas_2021_13,wasvsdal_2021_14,phivswas_2021_15,dalvswas_2021_16,wasvsphi_2021_17,nygvswas_2021_18
home_team,den,was,car,was,lv,was,phi,dal,was,nyg
away_team,was,tb,was,sea,was,dal,was,was,phi,was
year,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021
week,8,10,11,12,13,14,15,16,17,18
wl_total,2.0,2.0,3.0,4.0,5.0,6.0,6.0,6.0,6.0,6.0
game_count,7.0,8.0,9.0,10.0,11.0,12.0,13.0,14.0,15.0,16.0
avg_pf,20.857143,19.5,20.555556,21.2,20.818182,20.5,20.461538,20.214286,19.8,19.5625
avg_pa,30.0,28.375,27.333333,26.7,25.636364,24.75,24.923077,25.071429,27.133333,26.6875
avg_point_diff,-9.142857,-8.875,-6.777778,-5.5,-4.818182,-4.25,-4.461538,-4.857143,-7.333333,-7.125
